In [1]:
import numpy as np
from scipy.optimize import minimize

In [ ]:
def preprocess(fname):
    machines = []
    with open(fname, "r") as f:
        line = f.readline()
        while line:
            if "Button A" in line:
                temp = [x.split("+") for x in line.strip().split(",")]
                A = np.array([temp[0][-1], temp[1][-1]], dtype=np.int64)
            elif "Button B" in line:
                temp = [x.split("+") for x in line.strip().split(",")]
                B = np.array([temp[0][-1], temp[1][-1]], dtype=np.int64)
            elif "Prize" in line:
                temp = [x.split("=") for x in line.strip().split(",")]
                p = np.array([temp[0][-1], temp[1][-1]], dtype=np.int64)
                machines.append({"A": A,
                                 "B": B,
                                 "p": p})
                
            line = f.readline()

    return machines

def part1(machines):
    cost = 0
    Apress = np.arange(0,101)
    Bpress = np.arange(0,101)

    for m in machines:
        Acomb = np.outer(Apress, m["A"])
        Bcomb = np.outer(Bpress, m["B"])
        for Ai, _ in enumerate(Acomb):
            total = Acomb[Ai] + Bcomb
            found = np.all(total == m["p"], axis=1).nonzero()
            if np.any(found[0]):
                cost += 3 * Apress[Ai] + Bpress[found[0]]
                break

    return cost

machines = preprocess("day13_example.txt")
part1_example_sol = part1(machines)
print(f"Part 1 solution for example data: {(part1_example_sol)}")
assert (part1_example_sol) == 480

In [ ]:
machines = preprocess("day13_input.txt")
part1_sol = part1(machines)
print(f"Part 1 solution: {(part1_sol)}")


In [ ]:
part1_example_sol = part1(machines)
print(f"Part 1 solution for example data: {(part1_example_sol)}")

In [ ]:
def error(x, A, B, p):
    na, nb = x
    return np.sum((na*A + nb*B - p)**2)

def part2(machines):
    cost = 0
    offset = 10000000000000
    for m in machines:
        m["p"] = m["p"] + offset
        x0 = offset/(0.5 * m["A"] + 0.5 * m["B"])
        res = minimize(error, x0, method='nelder-mead', args=(m["A"], m["B"], m["p"]), tol=1e-7, options={"maxiter": 10_000})
        int_sol = np.round(res.x)
        err = error(int_sol, m["A"], m["B"], m["p"])
        if err == 0:
            cost += 3 * int_sol[0] + int_sol[1]
    return cost

machines = preprocess("day13_input.txt")
part2_sol = part2(machines)
print(f"Part 2 solution: {(part2_sol)}")